In [44]:
import pandas as pd
import numpy as np 
import seaborn as sns
from matplotlib import pyplot as plt
sns.set()


In [ ]:
#import 2020 QB stats CSV
DF2020 = pd.read_csv('supp/2020QB.csv')
#splitting on '\' to keep player name
Name = DF2020['Player'].str.split('\\').str[0]
Name = Name.values.tolist()
Name = Name
#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
#needed to add .strip() to lagging white space which affected merge downstream
Name = [remove_punc(i).strip() for i in Name]
DF2020['Player Name'] = Name
DF2020['Pos'] = DF2020['Pos'].str.upper()
DF2020 = DF2020.iloc[:, 2:]
DF2020['Year'] = '2020'



In [15]:
#import 2019 QB stats CSV
DF2019 = pd.read_csv('supp/2019QB.csv')
#splitting on '\' to keep player name
Name = DF2019['Player'].str.split('\\').str[0]
Name = Name.values.tolist()
Name = Name
#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
#needed to add .strip() to lagging white space which affected merge downstream
Name = [remove_punc(i).strip() for i in Name]
DF2019['Player Name'] = Name
DF2019['Pos'] = DF2019['Pos'].str.upper()
DF2019 = DF2019.iloc[:, 2:]
DF2019['Year'] = '2019'



In [10]:
#import 2018 QB stats CSV
DF2018 = pd.read_csv('supp/2019QB.csv')
#splitting on '\' to keep player name
Name = DF2018['Player'].str.split('\\').str[0]
Name = Name.values.tolist()
Name = Name
#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
#needed to add .strip() to lagging white space which affected merge downstream
Name = [remove_punc(i).strip() for i in Name]
DF2018['Player Name'] = Name
DF2018['Pos'] = DF2018['Pos'].str.upper()
DF2018 = DF2018.iloc[:, 2:]
DF2018['Year'] = '2018'

In [16]:
#concat DF here with fields I want
DF_QB = pd.concat([DF2020,DF2019,DF2018])
DF_QB = DF_QB[['Player Name', 'Age', 'Pos', 'Tm', 'Year', 'G', 'GS', 'Cmp', 'Att', 'Yds', 'TD']]


In [19]:
DFPM = pd.read_csv('supp/playermeasureables - Sheet1.csv')
Last_NamePM = DFPM['Player'].str.split(', ').str[0]
Last_NamePM = Last_NamePM.values.tolist()
First_NamePM = DFPM['Player'].str.split(', ').str[1]
First_NamePM = First_NamePM.values.tolist()
DFPM['First Name'] = First_NamePM
DFPM['Last Name'] = Last_NamePM
DFPM['Player Name'] = DFPM[['First Name', 'Last Name']].apply(lambda x: ' '.join(x), axis=1)
DFPM_Info = DFPM[['Player Name', 'Pos', 'Team', 'Height', 'Weight', 'DOB']]
#DFPM_Info.loc[DFPM_Info['Player Name']=='Patrick Mahomes']



In [41]:
#DF_QB.merge(DFPM_Info, how='left', on='Player Name')
DF_QB_PM = pd.merge(DF_QB, DFPM_Info, how='left', on=['Player Name', 'Pos'])
DF_QB_PM = DF_QB_PM[DF_QB_PM['Att'] > 60]
DF_QB_PM.head(5)

,Player Name,Age,Pos,Tm,Year,G,GS,Cmp,Att,Yds,TD,Team,Height,Weight,DOB
0,Matt Ryan,35,QB,ATL,2020,16,16,407,626,4581,26,ATL,"6'4""",217 lbs,5/17/85
1,Tom Brady,43,QB,TAM,2020,16,16,401,610,4633,40,TB,"6'4""",225 lbs,8/3/77
2,Ben Roethlisberger,38,QB,PIT,2020,15,15,399,608,3803,33,PIT,"6'5""",240 lbs,3/2/82
3,Justin Herbert,22,QB,LAC,2020,15,15,396,595,4336,31,LAC,"6'6""",237 lbs,3/10/98
4,Patrick Mahomes,25,QB,KAN,2020,15,15,390,588,4740,38,KC,"6'3""",230 lbs,9/17/95


In [48]:
#2018 merged data
DF_QB_PM_2018 = DF_QB_PM[DF_QB_PM['Year'] == '2018']

#2019 merged data
DF_QB_PM_2019 = DF_QB_PM[DF_QB_PM['Year'] == '2019']
#2020 merged data
DF_QB_PM_2020 = DF_QB_PM[DF_QB_PM['Year'] == '2020']
DF_QB_PM_2018.head(30)


,Player Name,Age,Pos,Tm,Year,G,GS,Cmp,Att,Yds,TD,Team,Height,Weight,DOB
214,Jared Goff,25,QB,LAR,2018,16,16,394,626,4638,22,LAR,"6'4""",222 lbs,10/14/94
215,Jameis Winston,25,QB,TAM,2018,16,16,380,626,5109,33,NO,"6'4""",231 lbs,1/6/94
216,Matt Ryan,34,QB,ATL,2018,15,15,408,616,4466,26,ATL,"6'4""",217 lbs,5/17/85
217,Tom Brady,42,QB,NWE,2018,16,16,373,613,4057,24,TB,"6'4""",225 lbs,8/3/77
218,Carson Wentz,27,QB,PHI,2018,16,16,388,607,4039,27,IND,"6'5""",237 lbs,12/30/92
219,Dak Prescott,26,QB,DAL,2018,16,16,388,596,4902,30,DAL,"6'2""",238 lbs,7/29/93
220,Philip Rivers,38,QB,LAC,2018,16,16,390,591,4615,23,IND,"6'5""",228 lbs,12/8/81
221,Aaron Rodgers,36,QB,GNB,2018,16,16,353,569,4002,26,GB,"6'2""",225 lbs,12/2/83
222,Kyler Murray,22,QB,ARI,2018,16,16,349,542,3722,20,ARI,"5'10""",207 lbs,8/7/97
223,Baker Mayfield,24,QB,CLE,2018,16,16,317,534,3827,22,CLE,"6'1""",215 lbs,4/14/95


In [58]:
# fig, axes = plt.subplots(1, 3, figsize=(20,10))
# fig.suptitle('Completion Distribution by Height')
# axes[0].set_title('2018')
# axes[1].set_title('2019')
# axes[2].set_title('2020')
#2018
plt.figure(figsize=(8,8))
sns.displot(DF_QB_PM_2018['Height'])
# sns.histplot(ax=axes[0], x=DF_QB_PM[DF_QB_PM['Height']], y=DF_QB_PM[DF_QB_PM['Cmp']]
# axes[0].set_title('2018')


ConversionError: Failed to convert value(s) to axis units: 0      6'4"
1      6'4"
2      6'4"
3      6'4"
4      6'5"
5      6'2"
6      6'5"
7      6'2"
8     5'10"
9      6'1"
10     6'2"
11     6'2"
12    5'11"
13     6'3"
14     6'2"
15     6'2"
16     6'3"
17     6'3"
18     6'2"
19     6'1"
20     6'5"
21     6'5"
22     6'4"
23     6'3"
24     6'3"
25     6'2"
26     6'0"
27     6'3"
28     6'4"
29     6'5"
30      NaN
31      NaN
32      NaN
33     6'2"
34     6'1"
35      NaN
36     6'4"
37     6'4"
38      NaN
39      NaN
40      NaN
41      NaN
42      NaN
43      NaN
44      NaN
45      NaN
46      NaN
47      NaN
48      NaN
49      NaN
50      NaN
Name: x, dtype: object